# LLM SEDERHANA

In [1]:
pip install langchain openai matplotlib pandas


Note: you may need to restart the kernel to use updated packages.


## 1


In [3]:
import pandas as pd
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Buat data penjualan langsung di kode
data = {
    "tanggal": [
        "2025-06-01", "2025-06-02", "2025-06-03",
        "2025-06-04", "2025-06-05", "2025-06-06"
    ],
    "produk": [
        "Produk A", "Produk B", "Produk A",
        "Produk C", "Produk B", "Produk A"
    ],
    "jumlah_terjual": [100, 80, 120, 50, 90, 110],
    "diskon_persen": [10, 5, 15, 0, 10, 20]
}

df = pd.DataFrame(data)
df["tanggal"] = pd.to_datetime(df["tanggal"])

# Tool untuk hitung korelasi saja
def sales_correlation_tool(query: str) -> str:
    corr = df["diskon_persen"].corr(df["jumlah_terjual"])
    return f"{corr}"

tools = [
    Tool(
        name="SalesCorrelation",
        func=sales_correlation_tool,
        description="Hitung korelasi antara diskon dan jumlah terjual."
    )
]

# Inisialisasi LLM
llm = ChatOpenAI(temperature=0)

# Prompt template untuk interpretasi korelasi
template = """
Hasil korelasi antara diskon dan jumlah penjualan adalah {correlation}.
Jelaskan arti dari nilai korelasi ini secara singkat, apakah positif atau negatif,
dan apa artinya untuk penjualan.
Berikan contoh penjelasan yang mudah dimengerti.
"""

prompt = PromptTemplate(
    input_variables=["correlation"],
    template=template
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# Initialize agent dengan tool
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# 1. Panggil agent untuk hitung korelasi (Action + Observation)
correlation_str = agent.run("Hitung korelasi diskon dan penjualan.")

# 2. Beri korelasi ke LLMChain untuk interpretasi dan penjelasan
response = llm_chain.run(correlation=correlation_str)

print("Interpretasi AI:")
print(response)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9824\2825341156.py:54: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)




> Entering new AgentExecutor chain...
I should use the SalesCorrelation tool to calculate the correlation between discount and sales.
Action: SalesCorrelation
Action Input: "diskon dan penjualan"
Observation: 0.9111916759223988
Thought:I now know the final answer
Final Answer: 0.9111916759223988

> Finished chain.
Interpretasi AI:
Nilai korelasi antara diskon dan jumlah penjualan sebesar 0.9111916759223988 menunjukkan adanya hubungan positif yang kuat antara kedua variabel tersebut. Artinya, semakin besar diskon yang diberikan, maka jumlah penjualan juga cenderung meningkat.

Contoh penjelasan yang mudah dimengerti adalah ketika sebuah toko memberikan diskon besar untuk produk tertentu, maka kemungkinan besar jumlah penjualan produk tersebut akan meningkat secara signifikan. Hal ini dapat terjadi karena konsumen cenderung tertarik untuk membeli produk dengan diskon yang menarik. Sebaliknya, jika diskon yang diberikan kecil atau tidak ada, maka jumlah penjualan juga cenderung stagnan 

##  2

In [4]:
import pandas as pd
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Contoh data pelanggan
data = {
    "customer_id": [1,2,3,4,5,6,7,8,9,10],
    "gender": ["M", "F", "F", "M", "M", "F", "F", "M", "F", "M"],
    "purchase_datetime": [
        "2025-06-01 10:00", "2025-06-01 11:30", "2025-06-02 09:15",
        "2025-06-02 18:45", "2025-06-03 14:00", "2025-06-03 20:30",
        "2025-06-04 13:00", "2025-06-04 17:00", "2025-06-05 19:00", "2025-06-05 21:15"
    ],
    "purchase_amount": [100, 150, 80, 200, 50, 90, 120, 60, 130, 170],
    "product": ["A","B","A","C","B","A","C","B","A","C"]
}

df = pd.DataFrame(data)
df["purchase_datetime"] = pd.to_datetime(df["purchase_datetime"])

# Tool untuk analisis data manual
def sales_detailed_analysis(query: str) -> str:
    avg_purchase_gender = df.groupby("gender")["purchase_amount"].mean()
    purchase_count_by_hour = df.groupby(df["purchase_datetime"].dt.hour).size()
    peak_hour = purchase_count_by_hour.idxmax()
    gender_peak_hour = df[df["purchase_datetime"].dt.hour == peak_hour].groupby("gender").size()
    most_active_gender = gender_peak_hour.idxmax()

    result = (
        f"Rata-rata pembelian berdasarkan jenis kelamin:\n{avg_purchase_gender.to_string()}\n\n"
        f"Jumlah pembelian per jam:\n{purchase_count_by_hour.to_string()}\n\n"
        f"Jam sibuk: {peak_hour}:00\n"
        f"Jenis kelamin paling aktif di jam sibuk: {most_active_gender}"
    )
    return result

tools = [
    Tool(
        name="SalesDetailedAnalyzer",
        func=sales_detailed_analysis,
        description="Analisis data penjualan secara detail dan spesifik."
    )
]

llm = ChatOpenAI(temperature=0)

prompt_template = """
Berikut adalah hasil analisis data penjualan:

{analysis}

Tolong jelaskan hasil analisis ini dengan bahasa yang mudah dimengerti,
dan berikan saran strategis yang relevan untuk bisnis berdasarkan hasil tersebut.
"""

prompt = PromptTemplate(input_variables=["analysis"], template=prompt_template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

#  hasil analisis
analysis_result = agent.run("Lakukan analisis detail penjualan.")

#LLM untuk penjelasan & saran
response = llm_chain.run(analysis=analysis_result)

print("Hasil Analisis dan Saran dari AI:")
print(response)




> Entering new AgentExecutor chain...
Saya perlu menggunakan SalesDetailedAnalyzer untuk mendapatkan informasi yang spesifik.
Action: SalesDetailedAnalyzer
Action Input: "penjualan"
Observation: Rata-rata pembelian berdasarkan jenis kelamin:
gender
F    114.0
M    116.0

Jumlah pembelian per jam:
purchase_datetime
9     1
10    1
11    1
13    1
14    1
17    1
18    1
19    1
20    1
21    1

Jam sibuk: 9:00
Jenis kelamin paling aktif di jam sibuk: F
Thought:Saya perlu melihat lebih detail tentang pola pembelian pada jam 9:00.
Action: SalesDetailedAnalyzer
Action Input: "pembelian jam 9:00"
Observation: Rata-rata pembelian berdasarkan jenis kelamin:
gender
F    114.0
M    116.0

Jumlah pembelian per jam:
purchase_datetime
9     1
10    1
11    1
13    1
14    1
17    1
18    1
19    1
20    1
21    1

Jam sibuk: 9:00
Jenis kelamin paling aktif di jam sibuk: F
Thought:Saya sudah memiliki informasi yang cukup untuk menjawab pertanyaan.
Final Answer: Rata-rata pembelian berdasarkan jen

## 3

In [5]:
from openai import OpenAI

# client OpenAI 
client = OpenAI()

def analisis_sentimen(teks):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Kamu adalah analis sentimen yang jujur dan akurat."},
            {"role": "user", "content": f"Analisa sentimen dari kalimat berikut:\n\n{teks}"}
        ],
    )
    return response.choices[0].message.content

if __name__ == "__main__":
    contoh_review = [
        "Produk ini sangat bagus dan memuaskan!",
        "Pelayanan yang buruk, saya kecewa sekali.",
        "Barang sampai tepat waktu tapi kualitas biasa saja.",
        "Harga mahal tapi kualitas sesuai.",
        "Saya suka desainnya tapi ukurannya terlalu kecil."
    ]

    for review in contoh_review:
        hasil = analisis_sentimen(review)
        print(f"Review: {review}")
        print(f"Sentimen: {hasil}")
        print("-" * 40)


Review: Produk ini sangat bagus dan memuaskan!
Sentimen: Kalimat tersebut memiliki sentimen positif. Ungkapan "sangat bagus" dan "memuaskan" menunjukkan kepuasan dan apresiasi terhadap produk. Secara keseluruhan, kalimat ini mencerminkan pengalaman yang baik dengan produk tersebut.
----------------------------------------
Review: Pelayanan yang buruk, saya kecewa sekali.
Sentimen: Kalimat tersebut mengandung sentimen negatif. Ungkapan "pelayanan yang buruk" menunjukkan ketidakpuasan, dan frasa "saya kecewa sekali" memperkuat perasaan negatif tersebut. Secara keseluruhan, kalimat ini mencerminkan kekecewaan dan ketidakpuasan terhadap suatu layanan.
----------------------------------------
Review: Barang sampai tepat waktu tapi kualitas biasa saja.
Sentimen: Sentimen dari kalimat tersebut cenderung netral cenderung negatif. Poin positifnya adalah barang yang sampai tepat waktu, tetapi diimbangi dengan penilaian negatif mengenai kualitas yang dianggap biasa saja. Secara keseluruhan, kalim

## 4

In [7]:
import pandas as pd

data = {
    "review": [
        "Produk ini sangat bagus dan memuaskan!",
        "Pelayanan yang buruk, saya kecewa sekali.",
        "Barang sampai tepat waktu tapi kualitas biasa saja.",
        "Harga mahal tapi kualitas sesuai.",
        "Saya suka desainnya tapi ukurannya terlalu kecil."
    ]
}

df = pd.DataFrame(data)
df.to_csv("reviews.csv", index=False)


In [9]:
import pandas as pd
from openai import OpenAI
import time

# Inisialisasi client OpenAI
client = OpenAI()

def analisis_sentimen(teks):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Kamu adalah analis sentimen yang hanya menjawab dengan kata Positif, Negatif, atau Netral tanpa penjelasan."},
            {"role": "user", "content": f"Analisa sentimen dari kalimat berikut:\n\n{teks}"}
        ],
    )
    return response.choices[0].message.content.strip()


def proses_batch(input_csv, output_csv, kolom_teks):
    df = pd.read_csv(input_csv)
    hasil_sentimen = []

    for i, row in df.iterrows():
        teks = row[kolom_teks]
        sentimen = analisis_sentimen(teks)
        hasil_sentimen.append(sentimen)
        print(f"Memproses baris {i+1}/{len(df)}: {sentimen}")
        time.sleep(1)  # Delay supaya tidak kena rate limit

    df['sentimen'] = hasil_sentimen
    df.to_csv(output_csv, index=False)
    print(f"Hasil analisis disimpan di {output_csv}")

if __name__ == "__main__":
    input_file = "reviews.csv"     # File CSV input berisi kolom teks review
    output_file = "reviews_sentimen.csv"
    kolom_teks = "review"          # Nama kolom teks di CSV

    proses_batch(input_file, output_file, kolom_teks)


Memproses baris 1/5: Positif
Memproses baris 2/5: Negatif
Memproses baris 3/5: Netral
Memproses baris 4/5: Netral
Memproses baris 5/5: Netral
Hasil analisis disimpan di reviews_sentimen.csv


In [15]:
import pandas as pd
from openai import OpenAI
import time

# Inisialisasi client OpenAI
client = OpenAI()

def analisis_sentimen(teks):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Kamu adalah analis sentimen yang hanya menjawab dengan kata Positif, Negatif, atau Netral tanpa penjelasan."},
            {"role": "user", "content": f"Analisa sentimen dari kalimat berikut:\n\n{teks}"}
        ],
    )
    return response.choices[0].message.content.strip()

In [16]:
hasil = analisis_sentimen("Saya sangat kecewa dengan layanan ini.")
print("Output dari fungsi analisis_sentimen:")
print(hasil)


Output dari fungsi analisis_sentimen:
Negatif


In [12]:
def proses_batch(input_csv, output_csv, kolom_teks):
    df = pd.read_csv(input_csv)
    hasil_sentimen = []

    for i, row in df.iterrows():
        teks = row[kolom_teks]
        sentimen = analisis_sentimen(teks)
        hasil_sentimen.append(sentimen)
        print(f"Memproses baris {i+1}/{len(df)}: {sentimen}")
        time.sleep(1)  

    df['sentimen'] = hasil_sentimen
    df.to_csv(output_csv, index=False)
    print(f"Hasil analisis disimpan di {output_csv}")

In [ ]:
if __name__ == "__main__":
    input_file = "reviews.csv"     
    output_file = "reviews_sentimen.csv"
    kolom_teks = "review"         
    proses_batch(input_file, output_file, kolom_teks)

In [2]:
pip install langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI
import pandas as pd

# Baca data
df = pd.read_csv("player_stats.csv")  

# Inisialisasi Agent dengan izin eksekusi kode
agent = create_pandas_dataframe_agent(
    llm=OpenAI(temperature=0),
    df=df,
    verbose=True,
    allow_dangerous_code=True 
)

# Ajukan pertanyaan
pertanyaan = "Siapa pemain dengan gol terbanyak? Berikan nama, klub, dan jumlah golnya."
hasil = agent.run(pertanyaan)
print(hasil)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14916\3379355741.py:18: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  hasil = agent.run(pertanyaan)




> Entering new AgentExecutor chain...
Thought: I need to find the player with the highest number of goals.
Action: python_repl_ast
Action Input: df.loc[df['Goals'].idxmax()]Player Name        Mohamed Salah
Club                   Liverpool
Nationality                Egypt
Position                     MID
Appearances                   38
Minutes                     3377
Goals                         29
Assists                       18
Shots                        130
Shots On Target               22
Name: 324, dtype: objectI need to extract the player name, club, and number of goals from this result.
Action: python_repl_ast
Action Input: df.loc[df['Goals'].idxmax()]['Player Name'], df.loc[df['Goals'].idxmax()]['Club'], df.loc[df['Goals'].idxmax()]['Goals']('Mohamed Salah', 'Liverpool', np.int64(29))I now know the final answer.
Final Answer: Mohamed Salah, Liverpool, 29 goals.

> Finished chain.
Mohamed Salah, Liverpool, 29 goals.


In [7]:
import pandas as pd

df = pd.read_csv("player_stats.csv")

# 1. Pemain dengan gol terbanyak
top_scorer = df[df["Goals"] == df["Goals"].max()]
print("Top Scorer:", top_scorer[["Player Name", "Club", "Goals"]].to_string(index=False))

# 2. Pemain dengan menit bermain terbanyak (top 3)
most_minutes = df.sort_values("Minutes", ascending=False).head(3)
print("\nMost Minutes Played:")
print(most_minutes[["Player Name", "Club", "Minutes"]].to_string(index=False))

Top Scorer:   Player Name      Club  Goals
Mohamed Salah Liverpool     29

Most Minutes Played:
   Player Name              Club  Minutes
     Matz Sels Nottingham Forest     3420
Dean Henderson    Crystal Palace     3420
    David Raya           Arsenal     3420


In [8]:
df.loc[df['Goals'].idxmax()]['Player Name'], df.loc[df['Goals'].idxmax()]['Club'], df.loc[df['Goals'].idxmax()]['Goals']

('Mohamed Salah', 'Liverpool', np.int64(29))

In [9]:
pip install transformers torch sentencepiece

Note: you may need to restart the kernel to use updated packages.


pipeline

In [14]:
from transformers import pipeline

# Inisialisasi pipeline analisis sentimen
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"  # Model default untuk English
)

# Contoh teks untuk dianalisis
texts = [
    "I love this product! It's amazing.",
    "The service was terrible and slow.",
    "It's okay, but could be better."
]

# Analisis sentimen
results = sentiment_analyzer(texts)

# Tampilkan hasil
for text, result in zip(texts, results):
    print(f"Text: {text}")
    print(f"Sentimen: {result['label']} (Score: {result['score']:.4f})\n")

Device set to use cpu


Text: I love this product! It's amazing.
Sentimen: POSITIVE (Score: 0.9999)

Text: The service was terrible and slow.
Sentimen: NEGATIVE (Score: 0.9996)

Text: It's okay, but could be better.
Sentimen: POSITIVE (Score: 0.9623)



tokenezer

In [8]:
import tiktoken
encoder = tiktoken.encoding_for_model("gpt-4")

# Contoh teks
text = "Selamat pagi, apa kabar?"

token_ids = encoder.encode(text)
print("Token IDs:", token_ids)
print("Jumlah Token:", len(token_ids))

print("Teks Asli:", encoder.decode(token_ids))

Token IDs: [40141, 43011, 15117, 72, 11, 61693, 75095, 277, 30]
Jumlah Token: 9
Teks Asli: Selamat pagi, apa kabar?
